In [1]:
# 다른 경로에 있는 모듈을 가져오기 위해 절대 경로 추가

import os
import sys

root = os.path.join(os.path.abspath(os.getcwd()), '..', '..')
root = os.path.normpath(root)

if root not in sys.path:
    sys.path.insert(0, root)

In [2]:
# 에브리타임 게시글의 제목과 내용을 가져옴

from src.utils import db

connection = db.get_connection()
cursor = connection.cursor()

cursor.execute('SELECT * FROM `everytime_articles`')
articles = cursor.fetchall()
contents = [(article['title'] + ' ' + article['text']).replace('\n', ' ') for article in articles]

Connected to DB successfully.


In [3]:
# 형태소 분석기가 멈출 수 있으므로 너무 긴 문장은 제외

contents = [content for content in contents if len(content) < 2000]

In [4]:
# 가져온 데이터의 일부를 출력

for content in contents[:20]:
    print(content)

7급 지역인재 추천 이거 아는 사람? 지역인재 추천으로 7급 추천 받을 수 있는 거던데 이거에 대해 아는 사람 있음? 전산개발 생각하고있는데
전정대 3학년 되는 학생인데요 최근 앱개발에 관심이 생겨서 알아보고 있는데요, 혹시 앱개발과 반도체 관련 전공을 둘다 필요로 하는 부서?직업? 같은게 있나요? 너무 다른 분야라고 생각이 돼서요..
혹시 컴활에 대해 물어보고 싶은분 물어보세요! 앙!
공설입 지원금 언제들어오냐...
나 너무 배고파 치킨 추천좀!!! 메뉴 이름도 알려줘!!!
50대 여성분한테 작별선물 뭐가 좋을까 같이 일하시는분한테 드리려고 하는데 되게 잘챙겨주시고 고마웠어서 뭐라도 하나 챙겨드리려고하는데 뭐가좋을까요? 부채같은거 생각했는데.. 괜찮은거 있을까요
자취생의 간단 삼겹살 정식 소소한 행복
문과는 올바른 가치관을 가져야 하고 이 사회를 공부해서 잘 운영해야하고 이과는 이 세상을 발전시켜야하는데 세상 꼬라지나 우리나라 꼬라지 보면 문과들은 공부 안하는게 맞다 기술은 발전하는데 인간들이 발전 못하네
채널돌리다 복면가왕봤는데 대놓고 소찬휘네 ㄲㅋㅋ
사람머리뼈 ㄷㄷ
공머생 말좀 묻겠수다 노트북 다들 뭐쓰쇼?
사는게 고민인 너네완 다르게 사는게 고민 하~!
학교근처에서 밥 배달시켜먹을라는데 추천좀! 배민이나 요기요에서 시킬라하는데 추천좀! 돈은 크게 상관없음
중화사상 옛날 중화사상 = 황제국, 상전 대접을 받는 대신 경제적, 군사적 지원을 조.공국들에게 넉넉하게 베푸는 암묵적 거래. 지금 "짱깨"의 중화사상 = 몽골 야만족 원나라식으로 힘으로 누르고 돈과 자원을 뺏어옴. 나중에 짱깨가 흔들리면 뒷통수 박살나게 됨. 그리고 중화사상은 유교적인것 이기도 해서 "중국은 최고다 - 조.공국들에게 군자로써 모범을 보여야한다"가 패시브로 있었는데 공산당이 집권하고 "중공" "짱깨"가 되면서 "유교"와 "군자로써 모범을 보여야 한다"가 빠지고 "중국은 최고다"만 남음. 오히려 경제적, 군사적 지원을 나눠주고 상전 대접을 받는 외교는 미국이 하는 방식이 됨.
개성있는 자

In [5]:
# 불용어 정보를 가져옴

with open('../../stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = set(line.strip() for line in file.readlines())

In [6]:
# KoNLPy 형태소 분석기를 이용해 명사를 추출하고 상위 100개를 출력

import time
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()
noun_frequency = {}
total_time = 0
iteration_count = 0

for content in tqdm(contents):
    start_time = time.time()
    nouns = []

    try:
        nouns = okt.nouns(content)
    except UnicodeDecodeError:
        continue
    except Exception as e:
        print(e)
        continue

    for noun in nouns:
        if len(noun) > 1 and noun not in stopwords:
            if noun in noun_frequency:
                noun_frequency[noun] += 1
            else:
                noun_frequency[noun] = 1

    end_time = time.time()
    total_time += end_time - start_time
    iteration_count += 1

average_time = total_time / iteration_count
average_speed = 1 / average_time

print(f'Average iteration speed: {average_speed:.2f}it/s')

top_nouns = sorted(noun_frequency.items(), key=lambda x: x[1], reverse=True)[:100]

for noun in top_nouns:
    print(noun)

100%|██████████| 298639/298639 [08:35<00:00, 579.41it/s] 


Average iteration speed: 582.83it/s
('교수', 29919)
('사람', 27352)
('학교', 23533)
('수업', 19301)
('시험', 18430)
('학기', 18338)
('학년', 15536)
('학점', 15113)
('오늘', 13930)
('강의', 13916)
('진짜', 13329)
('생각', 13221)
('공부', 13037)
('과제', 12358)
('지금', 11194)
('그냥', 10596)
('신청', 10266)
('가요', 9858)
('과목', 9803)
('전공', 9447)
('광운대', 9435)
('성적', 8963)
('정도', 8633)
('추천', 8167)
('문제', 8017)
('질문', 7507)
('대학', 7424)
('건가', 6545)
('친구', 6525)
('수강신청', 6453)
('대면', 5903)
('내일', 5885)
('기간', 5803)
('교양', 5785)
('이해', 5740)
('졸업', 5566)
('쪽지', 5510)
('학생', 5419)
('장학금', 5130)
('계절', 5119)
('제발', 5066)
('신분', 4943)
('도서관', 4939)
('재수강', 4931)
('정정', 4666)
('영어', 4629)
('남자', 4571)
('보고', 4542)
('휴학', 4462)
('수강', 4427)
('하나요', 4419)
('취업', 4398)
('이제', 4229)
('실험', 4228)
('알바', 4102)
('다시', 4101)
('관련', 4051)
('한번', 3955)
('인강', 3902)
('군대', 3896)
('여자', 3865)
('때문', 3858)
('에타', 3821)
('준비', 3813)
('수학', 3795)
('공지', 3776)
('아시', 3638)
('계속', 3611)
('제출', 3567)
('보통', 3553)
('확인', 3545)
('동아리', 3524)
('점

In [7]:
# khaiii 형태소 분석기를 이용해 명사를 추출하고 상위 100개를 출력

import time
from khaiii import KhaiiiApi
from tqdm import tqdm

api = KhaiiiApi()
noun_frequency = {}
total_time = 0
iteration_count = 0

for content in tqdm(contents):
    start_time = time.time()
    nouns = []

    try:
        for word in api.analyze(content):
            for morph in word.morphs:
                if morph.tag == 'NNG' or morph.tag == 'NNP':
                    noun = morph.lex
                    if len(noun) > 1 and noun not in stopwords:
                        nouns.append(noun)
    except Exception as e:
        print(e)
        continue

    for noun in nouns:
        if noun in noun_frequency:
            noun_frequency[noun] += 1
        else:
            noun_frequency[noun] = 1

    end_time = time.time()
    total_time += end_time - start_time
    iteration_count += 1

average_time = total_time / iteration_count
average_speed = 1 / average_time

print(f'Average iteration speed: {average_speed:.2f}it/s')

top_nouns = sorted(noun_frequency.items(), key=lambda x: x[1], reverse=True)[:100]

for noun in top_nouns:
    print(noun)

100%|██████████| 298639/298639 [03:44<00:00, 1331.14it/s]


Average iteration speed: 1338.84it/s
('사람', 24336)
('교수', 22249)
('학교', 19210)
('수업', 17918)
('시험', 16375)
('학년', 14438)
('신청', 14179)
('학점', 13413)
('오늘', 13385)
('생각', 12575)
('진짜', 12310)
('ㅠㅠ', 12249)
('수강', 11863)
('공부', 11657)
('학기', 11382)
('과제', 11200)
('가능', 8928)
('전공', 8651)
('강의', 8422)
('과목', 8222)
('문제', 7474)
('대학', 7184)
('질문', 6692)
('친구', 6627)
('정도', 6251)
('추천', 5980)
('내일', 5945)
('졸업', 5631)
('학생', 5468)
('ㅈㄱㄴ', 5402)
('교양', 5270)
('ㅋㅋ', 5198)
('기간', 5149)
('광운대', 4781)
('부탁', 4602)
('대면', 4459)
('성적', 4339)
('여자', 4298)
('이해', 4269)
('남자', 4257)
('장학금', 4243)
('도서관', 4152)
('취업', 4135)
('휴학', 4086)
('공지', 4067)
('수학', 4036)
('기말', 3849)
('공학', 3841)
('관련', 3752)
('감사', 3646)
('실험', 3593)
('복학', 3539)
('군대', 3496)
('정정', 3436)
('시작', 3430)
('필요', 3371)
('원래', 3334)
('준비', 3327)
('기숙사', 3291)
('요즘', 3262)
('ㅜㅜ', 3251)
('영어', 3180)
('중간', 3167)
('인강', 3142)
('제출', 3139)
('확인', 3089)
('쪽지', 3082)
('처음', 3064)
('근처', 2981)
('지금', 2971)
('내용', 2940)
('계절', 2880)
('ㅋㅋㅋ'